## Loading the dataset

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

# Define a function to change the MNIST label dimension
def change_to_right(wrong_labels):
    right_labels=[]
    for x in wrong_labels:
        for i in range(0,len(wrong_labels[0])):
            if x[i]==1:
                right_labels.append(i)
    return right_labels

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Convert class vectors to right format
y_train = tf.convert_to_tensor(np.array(change_to_right(keras.utils.to_categorical(y_train, num_classes))))
y_test = tf.convert_to_tensor(np.array(change_to_right(keras.utils.to_categorical(y_test, num_classes))))

2024-04-02 15:12:07.741093: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 15:12:07.748887: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 15:12:07.777344: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 15:12:07.777367: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 15:12:07.777384: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

## Hyperparameters

In [2]:

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 2000
num_epochs = 30

In [3]:
image_size = 56#28  # We'll resize input images to this size
patch_size = 28#14  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 192#96
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 16
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier


## Data augmentation

In [4]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        #layers.Resizing(image_size, image_size),
        #layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)

In [5]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

## Run

In [6]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

Epoch 1/30
27/27 [==============================] - 79s 3s/step - loss: 1.5722 - accuracy: 0.5735 - top-5-accuracy: 0.8984 - val_loss: 0.3390 - val_accuracy: 0.8973 - val_top-5-accuracy: 0.9953
Epoch 2/30
27/27 [==============================] - 71s 3s/step - loss: 0.4352 - accuracy: 0.8665 - top-5-accuracy: 0.9927 - val_loss: 0.1896 - val_accuracy: 0.9427 - val_top-5-accuracy: 0.9978
Epoch 3/30
27/27 [==============================] - 70s 3s/step - loss: 0.2845 - accuracy: 0.9148 - top-5-accuracy: 0.9960 - val_loss: 0.1393 - val_accuracy: 0.9610 - val_top-5-accuracy: 0.9975
Epoch 4/30
27/27 [==============================] - 70s 3s/step - loss: 0.2171 - accuracy: 0.9346 - top-5-accuracy: 0.9973 - val_loss: 0.1142 - val_accuracy: 0.9672 - val_top-5-accuracy: 0.9978
Epoch 5/30
27/27 [==============================] - 70s 3s/step - loss: 0.1832 - accuracy: 0.9459 - top-5-accuracy: 0.9978 - val_loss: 0.0997 - val_accuracy: 0.9723 - val_top-5-accuracy: 0.9983
Epoch 6/30
27/27 [============

## Possibility of Quantum Vision Transformer

The vision transformer might be enhanced by employing quantum architecture to form the quantum transformers. The quantum transformers are based on compound matrices, and can be built by using shallow quantum circuit. There are three already proposed quantum transformers, they are called as the orthogonal patch-wise neural network, quantum orthogonal transformer, and quantum compound transformer. 

### Orthogonal patch-wise Neural Network

This architecture is a transformer with a trivial attention mechanism, this is because each patch only pays attention to itself. Each input patch is multiplied by the same trainable matrix $V$ and one circuit per patch is used. There are $N = d$ qubits in each circuit and a quantum state with a vector data loader is used to encode each patch $x_i$. The multiplication of each patch with the matrix $V$ is done by a quantum orthogonal layer. Therefore, the output of each circuit is a quantum state encoding $Vx_i$

### Quantum Orthogonal Transformer

In this architecture, each attention coefficient $A_{ij} = x_i^TWx_j$ is firstly calculated; Each patch $x_j$ is loaded into the circuit with a vector loader followed by a trainable quantum orthogonal layer $W$, and then an inverse data loader of $x_i$ is applied. The result is the probability of measuring $1$ on the first qubit is $|x_i^TWx_j|^2 = A_{ij}^2$.

### Quantum Compound Transformer

The compound transformer works by firstly loading all patches with the same weight, applying superposition to each patch, and then applying orthogonal layer. By doing this, the features from each patch can be extracted and the patches can be re-weighted. The output will be computed as a weighted sum of the features extracted from all patches. 

### References

1. Gros, Claudius. "Visual Transformer and the MNIST Dataset." Institute for Theoretical Physics, Goethe University Frankfurt, 2023, https://itp.uni-frankfurt.de/~gros/StudentProjects/WS22_23_VisualTransformer/ <br/>
2. Kerenidis, Iordanis, et al. "Quantum Vision Transformers." (2022).
   
